Notebook for processing videos on Google Colab.
Note that even with a T4 GPU a single 5 seconds video easily takes 20+ minutes to process.

In [1]:
# Clone the main repo
%%capture
! git clone https://github.com/shubham-goel/4D-Humans.git 4D-Humans
%cd 4D-Humans

In [ ]:
!pip install torch
!pip install -e .[all]
!wget https://github.com/classner/up/raw/master/models/3D/basicModel_neutral_lbs_10_207_0_v1.0.0.pkl
!mkdir data/
!mv basicModel_neutral_lbs_10_207_0_v1.0.0.pkl data/

In [ ]:
!pip install git+https://github.com/brjathu/PHALP.git

In [ ]:
!pip install opencv-python

In [6]:
%cd 4D-Humans

[Errno 2] No such file or directory: '4D-Humans'
/content/4D-Humans


In [8]:
# Display the reconstruction video
def show_local_mp4_video(file_name, width=640, height=480):
  import io
  import base64
  from IPython.display import HTML
  video_encoded = base64.b64encode(io.open(file_name, 'rb').read())
  return HTML(data='''<video width="{0}" height="{1}" alt="test" controls>
                      <source src="data:video/mp4;base64,{2}" type="video/mp4" />
                      </video>'''.format(width, height, video_encoded.decode('ascii')))

#!ffmpeg -y -hide_banner -loglevel error -i outputs/PHALP_gymnasts.mp4 outputs/PHALP_gymnasts_ffmpeg.mp4
#show_local_mp4_video('outputs/PHALP_gymnasts_ffmpeg.mp4', width=960, height=540)

In [9]:
import os
from google.colab import drive
drive.mount('/content/drive')

video_dir = "drive/MyDrive/Project 24/Videos/"
input_dir = video_dir + "normal/"
output_dir = video_dir + "normal-outputs/"

Mounted at /content/drive


In [10]:
%cd ..

/content


In [ ]:
!ls drive/MyDrive/

In [ ]:
assert os.path.exists(input_dir)
assert os.path.exists(output_dir)

video_files = [f for f in os.listdir(input_dir) if f.startswith('normal-') and f.endswith('.mp4')]

# Sort the list of video files to make the assignment deterministic
video_files.sort()

# Split the sorted list into 4 batches
batch_size = len(video_files) // 4
batches = [video_files[i:i + batch_size] for i in range(0, len(video_files), batch_size)]

for i, batch in enumerate(batches):
    print(f"Batch {i + 1}:")
    for file in batch:
        print(file)
    print()

In [20]:
# Sometimes the model fails and you need to try again
# Keep track of the files that have already been processed
normal_batches = {
    "batch1": [
        "normal-DTU-side.mp4",
        "normal-DTU1-back.mp4",
        "normal-DTU1-front.mp4",
        "normal-DTU1-side.mp4",
        "normal-DTU2-back.mp4",
        "normal-DTU3-back.mp4",
        "normal-DTU3-side.mp4",
        "normal-DTU4-back.mp4",
        "normal-DTU4-side.mp4",
    ],
    "batch2": [
        "normal-angry_girlfriend-front.mp4",
        "normal-angry_girlfriend-side.mp4",
        "normal-athletic_male-front.mp4",
        "normal-athletic_male-side.mp4",
        "normal-figure_eight_walking-mixed.mp4",
        "normal-guy-side.mp4",
        "normal-happy_man-side.mp4",
        "normal-holly-back.mp4",
        "normal-holly-front.mp4",
    ],
    "batch3": [
        "normal-jake-back.mp4",
        "normal-jake-front.mp4",
        "normal-jake-side.mp4",
        "normal-jake-side2.mp4",
        "normal-karolina-frontback.mp4",
        "normal-london1-back.mp4",
        "normal-london_man_back.mp4",
        "normal-london_two_people-back.mp4",
        "normal-mafalda-back.mp4",
    ],
    "batch4": [
        "normal-mafalda-front.mp4",
        "normal-matured_woman-front.mp4",
        "normal-matured_woman-side.mp4",
        "normal-nerea-back.mp4",
        "normal-nerea-front.mp4",
        "normal-niklas-back.mp4",
        "normal-niklas-back2.mp4",
        "normal-niklas-front.mp4",
        "normal-niklas-front2.mp4",
        "normal-woman-side.mp4",
    ]
}

abnormal_batches = {
    "batch1": [
        "abnormal-antalgic_gait.mp4",
        "abnormal-ataxic_gait.mp4",
        "abnormal-cane1.mp4",
        "abnormal-circumduction_gait.mp4",
        "abnormal-drunk_woman-side_front.mp4",
    ],
    "batch2": [
        "abnormal-hemiplegic_gait .mp4",
        "abnormal-hemiplegic_gait2.mp4",
        "abnormal-hemiplegic_gait3.mp4",
        "abnormal-hemiplegic_gait4.mp4",
        "abnormal-limping1 (1).mp4",
    ],
    "batch3": [
        "abnormal-limping1.mp4",
        "abnormal-limping_11.mp4",
        "abnormal-limping_1_4.mp4",
        "abnormal-limping_2.mp4",
        "abnormal-limping_4.mp4",
    ],
    "batch4": [
        "abnormal-limping_gait1.mp4",
        "abnormal-limping_gait2.mp4",
        "abnormal-man.mp4",
        "abnormal-parkinsonian_gait.mp4",
        "abnormal-scissors_gait.mp4",
    ],
}

In [16]:
%cd 4D-Humans/

/content/4D-Humans


In [17]:
os.path.exists('../' + input_dir)

True

In [ ]:
def get_frame_count(video_path):
    import cv2

    video_capture = cv2.VideoCapture(video_path)
    # Get the total number of frames in the video
    total_frames = int(video_capture.get(cv2.CAP_PROP_FRAME_COUNT))
    video_capture.release()

    return total_frames

def process_video(file):
    if not os.path.exists(file):
        print("ERROR The file", file, "does not exist, skipping..")
        return

    maxFrames = max(0, get_frame_count(file) - 10)
    # Note the {} to pass python variables as arguments!
    # The '{file}' is used to pass a string as a argument
    # Will stil work without {} but does not produce any output files!
    !python track.py video.source='{file}' video.start_frame=5 video.end_frame={maxFrames}
    print('Done with file', file)

In [ ]:
batch = normal_batches['batch3']
input_files = ['../' + input_dir + fp for fp in batch]

for input in input_files:
    print('Processing video', input)
    #print(input)
    process_video(input)
    print('Done processing', input)

**Move the output to the Google Drive otherwise the output will be lost!**

In [ ]:
import shutil

def move_dir_contents(source_dir, target_dir):
    files_in_dir = os.listdir(source_dir) #[f for f in os.listdir(source_dir) if os.path.isfile(source_dir)]
    print('Found', len(files_in_dir), 'files to move')
    for file_name in files_in_dir:
        try:
            shutil.move(source_dir + file_name, target_dir+file_name)
            print('Success: Moved file', file_name, 'to', target_dir)
        except FileNotFoundError:
            print(f"Error: {file_name} not found in source_dir")
        except shutil.Error as e:
            print(f"Error: {file_name} could not be moved. Reason {e}")

# Move that b*tch
print('Moving /results/')
move_dir_contents('./outputs/results/', '../' + output_dir + 'results/')

print('\n\nMoving /_DEMO/')
move_dir_contents('./outputs/_DEMO/', '../' + output_dir + '_DEMO/')

print('\n\nMoving PHALP .mp4 files in /outputs/')
files_to_move = [f for f in os.listdir('./outputs/') if f.startswith('PHALP_') and f.endswith('mp4')]
print('Found', len(files_to_move), 'files to move')
for f in files_to_move:
    file_path = './outputs/' + f
    try:
        shutil.move('./outputs/' + f, '../' + output_dir + f)
        print('Success: Moved file', f, 'to', output_dir)
    except FileNotFoundError:
        print(f"Error: {f} not found in source_dir")
    except shutil.Error as e:
        print(f"Error: {f} could not be moved. Reason {e}")